Ceci est projet de classification binaire. Nous disposons de plusieurs jeux de données à exploiter afin de construire un modèle prédictif des résultats des partis politiques aux elections des USA à savoir : 
- Le parti Républicain (1)
- Le parti Démocrate (0) 

Nous utiliserons les données démographiques de la population pour créer un nouveau DataFrame pour le moèle, puis les résultats de l'élection de 2020 pour créer notre target. 

D'autres archives des elections antérieurs nous servirons aussi dans le processus de notre analyse exploratoire. 

In [9]:
# Import des bibliothèques 
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib as plt 

In [10]:
# Charger les Datasets 

Data_1 = "C:/Users/HP PROBOOK/Documents/Projet Datagong/Data education.csv"
Data_2 = "C:/Users/HP PROBOOK/Documents/Projet Datagong/Data population estimate.csv"
Data_3 = "C:/Users/HP PROBOOK/Documents/Projet Datagong/Data poverty estimate.csv"
Data_4 = "C:/Users/HP PROBOOK/Documents/Projet Datagong/Data Unemployment.xls - Unemployment Med HH Income.csv" 


In [11]:
# Charger les fichier dans un DataFrame 

df1 = pd.read_csv("C:/Users/HP PROBOOK/Documents/Projet Datagong/Data education.csv") 
df2 = pd.read_csv("C:/Users/HP PROBOOK/Documents/Projet Datagong/Data population estimate.csv") 
df3 = pd.read_csv("C:/Users/HP PROBOOK/Documents/Projet Datagong/Data poverty estimate.csv") 
df4 = pd.read_csv("C:/Users/HP PROBOOK/Documents/Projet Datagong/Data Unemployment.xls - Unemployment Med HH Income.csv") 

In [12]:
# Vérification des dimensions 

dataframes = [df1, df2, df3, df4]
noms = ['df1', 'df2', 'df3', 'df4']

for df, nom in zip(dataframes, noms):
    print(f"Dimensions de {nom} : {df.shape}")


Dimensions de df1 : (3283, 47)
Dimensions de df2 : (3273, 165)
Dimensions de df3 : (3193, 34)
Dimensions de df4 : (3275, 88)


In [13]:
# Suppression des lignes vides ou presque 

# On definit 80 comme seuil pour considérer une ligne comme presque vide
seuil_lignes = 0.8  

# Supprimer les lignes vides ou presque vides
for i, df in enumerate(dataframes):
    # Supprimer les lignes où toutes les valeurs sont NaN
    df.dropna(how='all', inplace=True)
    
    # Supprimer les lignes où un certain pourcentage des valeurs est NaN
    df.dropna(thresh=int(seuil_lignes * len(df.columns)), inplace=True)
    
    print(f"Dimensions de {noms[i]} après suppression des lignes vides : {df.shape}")


Dimensions de df1 après suppression des lignes vides : (3263, 47)
Dimensions de df2 après suppression des lignes vides : (3193, 165)
Dimensions de df3 après suppression des lignes vides : (3188, 34)
Dimensions de df4 après suppression des lignes vides : (3267, 88)


Avant de continuer, nous allons eliminer certaines variables pour eviter les duplications après la fusion. Pour ce fait, nous tiendrons compte uniquement des colonnes dans ce cas, qui contiennent le maximum d'informations. Du coup, nous allons calculer les pourcentages des valeurs manquantes par colonnes et garder celles qui presentent le moins de valeurs manqnuantes. 

In [14]:

# Fonction pour calculer le pourcentage de valeurs manquantes par colonne
def calculer_pourcentage_valeurs_manquantes(df):
    return df.isnull().mean() * 100

# Afficher le pourcentage de valeurs manquantes pour chaque DataFrame
for df, nom in zip(dataframes, noms):
    print(f"Pourcentage de valeurs manquantes pour {nom} :")
    print(calculer_pourcentage_valeurs_manquantes(df))
    print("\n")


Pourcentage de valeurs manquantes pour df1 :
FIPS Code                                                                   0.000000
State                                                                       0.000000
Area name                                                                   0.000000
Rural-urban Continuum Code 2003                                             1.593625
2003 Urban Influence Code                                                   1.593625
2013 Rural-urban Continuum Code                                             1.685565
2013 Urban Influence Code                                                   1.685565
Less than a high school diploma, 1970                                       2.513025
High school diploma only, 1970                                              2.513025
Some college (1-3 years), 1970                                              2.513025
Four years of college or higher, 1970                                       2.513025
Percent of adults wi

In [15]:

# Seuil pour le pourcentage de valeurs manquantes 
seuil_colonnes = 30

# Colonnes communes pour la fusion
colonnes_communes = ['FIPS Code', 'State', 'Area name']

# Filtrer les DataFrames pour ne garder que les colonnes avec moins de valeurs manquantes que le seuil
filtered_dataframes = []
for df, nom in zip(dataframes, noms):
    colonnes_a_garder = df.columns[df.isnull().mean() * 100 < seuil_colonnes]
    filtered_df = df[colonnes_a_garder]
    
    # Supprimer les espaces autour des noms de colonnes pour chaque DataFrame
    filtered_df.columns = filtered_df.columns.str.strip()
    
    # Vérifier la présence des colonnes communes après filtrage
    colonnes_presente = all(col in filtered_df.columns for col in colonnes_communes)
    if not colonnes_presente:
        print(f"Attention : Les colonnes communes ne sont pas toutes présentes dans {nom} après filtrage.")
        print(filtered_df.columns)
        
    # Ajout de vérifications supplémentaires
    print(f"Dimensions de {nom} avant ajout à filtered_dataframes : {filtered_df.shape}")
    
    filtered_dataframes.append(filtered_df)

# Vérifier les nouvelles dimensions des DataFrames filtrés
for i, df in enumerate(filtered_dataframes):
    print(f"Dimensions de {noms[i]} après filtrage : {df.shape}")

# Afficher les noms des colonnes des DataFrames filtrés
for df, nom in zip(filtered_dataframes, noms):
    print(f"Colonnes présentes dans {nom} après filtrage :")
    print(df.columns)
    print("\n")


Dimensions de df1 avant ajout à filtered_dataframes : (3263, 47)
Dimensions de df2 avant ajout à filtered_dataframes : (3193, 165)
Dimensions de df3 avant ajout à filtered_dataframes : (3188, 28)
Dimensions de df4 avant ajout à filtered_dataframes : (3267, 88)
Dimensions de df1 après filtrage : (3263, 47)
Dimensions de df2 après filtrage : (3193, 165)
Dimensions de df3 après filtrage : (3188, 28)
Dimensions de df4 après filtrage : (3267, 88)
Colonnes présentes dans df1 après filtrage :
Index(['FIPS Code', 'State', 'Area name', 'Rural-urban Continuum Code 2003',
       '2003 Urban Influence Code', '2013 Rural-urban Continuum Code',
       '2013 Urban Influence Code', 'Less than a high school diploma, 1970',
       'High school diploma only, 1970', 'Some college (1-3 years), 1970',
       'Four years of college or higher, 1970',
       'Percent of adults with less than a high school diploma, 1970',
       'Percent of adults with a high school diploma only, 1970',
       'Percent of adult

Nous pensons qu'il serait mieux d'imputer les valeurs manquantes dans chaque dataframe avant de les fusionner. Cette prcédure devrait amélorer davantage la qualité des données finales. Comme stratégie d'imputation, nous optons pour la moyenne pour les valeurs numériques et le mode pour les valeurs catégorielles. 

In [ ]:
# import de SimpleImputer 
from sklearn.impute import SimpleImputer 
# Stratégies d'imputation 

imputer_num = SimpleImputer(strategy='mean') 
imputer_cat = SimpleImputer(strategy='most_frequent') 

# Fonction pour imputer les valeurs manquantes par type de données 
def imputer_df(df): 
    df_num = df.select_dtypes(include=['float64', 'int64']) 
    df_cat = df.select_dtypes(exclude=['float64', 'int64']) 
    
    imputed_df_num = pd.DataFrame(imputer_num.fit_transform(df_num), columns=df_num.columns) 
    imputed_df_cat = pd.DataFrame(imputer_cat.fit_transform(df_cat), columns=df_cat.columns) 
    
    return pd.concat([imputed_df_num, imputed_df_cat], axis=1) 

# Imputer les valeurs manquantes dans chaque DataFrame filtré 
imputed_dataframes = [imputer_df(df) for df in filtered_dataframes] 

# Vérifier les nouvelles dimensions des DataFrames imputés 
for i, df in enumerate(imputed_dataframes): 
    print(f"Dimensions de {noms[i]} après imputation : {df.shape}") 

Dimensions de df1 après imputation : (3263, 47)
Dimensions de df2 après imputation : (3193, 165)
Dimensions de df3 après imputation : (3188, 28)
Dimensions de df4 après imputation : (3267, 88)


In [18]:
# Vérifier les valeurs manquantes après l'imputation
for df, nom in zip(imputed_dataframes, noms):
    print(f"Valeurs manquantes dans {nom} après imputation :")
    print(df.isnull().sum())
    print("\n")


Valeurs manquantes dans df1 après imputation :
FIPS Code                                                                   0
Rural-urban Continuum Code 2003                                             0
2003 Urban Influence Code                                                   0
2013 Rural-urban Continuum Code                                             0
2013 Urban Influence Code                                                   0
State                                                                       0
Area name                                                                   0
Less than a high school diploma, 1970                                       0
High school diploma only, 1970                                              0
Some college (1-3 years), 1970                                              0
Four years of college or higher, 1970                                       0
Percent of adults with less than a high school diploma, 1970                0
Percent of adults

A présent, nous pouvons passer à la fusion de nos dataframes !

In [19]:
# Définir une fonction pour fusionner les DataFrames
def fusionner_dataframes(dfs, on_columns, how='outer'):
    result = dfs[0]
    for i in range(1, len(dfs)):
        # Utiliser des suffixes personnalisés pour chaque fusion
        result = pd.merge(result, dfs[i], on=on_columns, how=how, suffixes=(f'_{i-1}', f'_{i}'))
    return result

# Fusionner les DataFrames sur les colonnes communes et nommer le DataFrame fusionné
Data_demograh = fusionner_dataframes(imputed_dataframes, colonnes_communes)

# Afficher les dimensions du DataFrame fusionné
print(f"Dimensions de Data_demograh : {Data_demograh.shape}")

# Afficher les premières lignes du DataFrame fusionné

Data_demograh.head()


Dimensions de Data_demograh : (6503, 319)


,FIPS Code,Rural-urban Continuum Code 2003_0,2003 Urban Influence Code,2013 Rural-urban Continuum Code,2013 Urban Influence Code,State,Area name,"Less than a high school diploma, 1970","High school diploma only, 1970","Some college (1-3 years), 1970",...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
0,0.0,5.050763,5.362504,4.925187,5.171758,US,United States,52 373 312,34 158 051,11 650 730,...,161 389 026,155 102 319,6 286 707,"3,9",163 100 055,157 115 247,5 984 808,"3,7",65 712,100
1,1000.0,5.050763,5.362504,4.925187,5.171758,AL,Alabama,1 062 306,468 269,136 287,...,2 216 627,2 130 845,85 782,"3,9",2 241 747,2 174 483,67 264,3,51 771,100
2,1001.0,2.000000,2.000000,2.000000,2.000000,AL,Autauga County,6 611,3 757,933,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1001.0,NaN,NaN,NaN,NaN,AL,"Autauga County, AL",NaN,NaN,NaN,...,26 196,25 261,935,"3,6",26 172,25 458,714,"2,7",58 233,"112,5"
4,1003.0,4.000000,5.000000,3.000000,2.000000,AL,Baldwin County,18 726,8 426,2 334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Vérifier les valeurs manquantes après la fusion
print("Valeurs manquantes dans Data_demograh après la fusion :")
print(Data_demograh.isnull().sum())


Valeurs manquantes dans Data_demograh après la fusion :
FIPS Code                                       0
Rural-urban Continuum Code 2003_0            3240
2003 Urban Influence Code                    3240
2013 Rural-urban Continuum Code              3240
2013 Urban Influence Code                    3240
                                             ... 
Employed_2019                                3236
Unemployed_2019                              3236
Unemployment_rate_2019                       3236
Median_Household_Income_2019                 3236
Med_HH_Income_Percent_of_State_Total_2019    3236
Length: 319, dtype: int64


In [21]:

# Imputation des valeurs manquantes dans le DataFrame final
imputer_final_num = SimpleImputer(strategy='mean')
imputer_final_cat = SimpleImputer(strategy='most_frequent')

def imputer_final_df(df):
    df_num = df.select_dtypes(include=['float64', 'int64'])
    df_cat = df.select_dtypes(exclude=['float64', 'int64'])
    
    imputed_df_num = pd.DataFrame(imputer_final_num.fit_transform(df_num), columns=df_num.columns)
    imputed_df_cat = pd.DataFrame(imputer_final_cat.fit_transform(df_cat), columns=df_cat.columns)
    
    return pd.concat([imputed_df_num, imputed_df_cat], axis=1)

# Appliquer l'imputation au DataFrame final
Data_demograh_imputed = imputer_final_df(Data_demograh)

# Afficher les dimensions du DataFrame après imputation
print(f"Dimensions de Data_demograh_imputed : {Data_demograh_imputed.shape}")

Data_demograh_imputed.head() 

Dimensions de Data_demograh_imputed : (6503, 319)


,FIPS Code,Rural-urban Continuum Code 2003_0,2003 Urban Influence Code,2013 Rural-urban Continuum Code,2013 Urban Influence Code,Rural-urban Continuum Code 2003_1,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2003_1,Urban_Influence_Code_2013_1,Economic_typology_2015,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
0,0.0,5.050763,5.362504,4.925187,5.171758,5.125598,5.008593,5.448518,5.267664,1.808402,...,161 389 026,155 102 319,6 286 707,"3,9",163 100 055,157 115 247,5 984 808,"3,7",65 712,100
1,1000.0,5.050763,5.362504,4.925187,5.171758,5.125598,5.008593,5.448518,5.267664,1.808402,...,2 216 627,2 130 845,85 782,"3,9",2 241 747,2 174 483,67 264,3,51 771,100
2,1001.0,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000,...,1 182,10 204,30,"3,6",2 671,2 693,284,"3,1",52 179,100
3,1001.0,5.050763,5.362504,4.925187,5.171758,5.125598,5.008593,5.448518,5.267664,1.808402,...,26 196,25 261,935,"3,6",26 172,25 458,714,"2,7",58 233,"112,5"
4,1003.0,4.000000,5.000000,3.000000,2.000000,4.000000,3.000000,5.000000,2.000000,5.000000,...,1 182,10 204,30,"3,6",2 671,2 693,284,"3,1",52 179,100
